**Similar to my first project. I am using more variables to create a complex, realistic picture of what income looks like.**

*This is data cleaning only. Statistical analysis was conducted in R.*

I filtered ACS PUMS 1-year data from 2018 for the columns ST, HISPEED, HINCP, PAP, GRPIP, FS, and FES.
  
The ST column is used to indicate the state from which the individual claims residence. I filtered the dataset for only rows with individuals from New York State. New York State’s numerical value in this column is ‘36’. 

The HISPEED column is used to indicate whether the individual has access to high-speed, broadband internet or not. The value options for this column were 0 indicating no paid internet access, 1 indicating access to broadband internet, and 2 indicating internet access but not broadband internet. I used all available values in this column.

The HINCP column is used to indicate the total income of the individual's household. I used all available values in this column, including the value’-60,000’ which was used to indicate group quarters.  

The FS column is used to indicate recipients of aid from the Supplemental Nutrition Assistance Program (SNAP). I filtered this column to remove N/A values. The values in this column are 1, indicating that the individual does receive assistance, and 2, indicating that the individual does not receive assistance. I replaced the 2 values with 0 so that the data would resemble a True or False format. 

The FES column gave details about how many parents in the household were employed. I filtered this data to exclude N/A values. There were 8 remaining columns detailing different living situations, however, I replaced these with 3 values. I used 0 to indicate neither parents are employed, 1 to indicate that one parent was employed, and 2 to indicate that both parents were employed. The PAP column indicates a value for how much public assistance income an individual receives. I filtered this column to exclude children younger than 15. 

Finally, the GRPIP column indicates the percentage of total income that the individual's rent makes up. I filtered this column to exclude people who were not paying rent.

In [18]:
#importing the necessary packages for data requisition 
import requests
import pandas as pd

def get_df(url):
    '''This is a function that retreived ACS PUMS data that I specified and inserts it into a DataFrame.
    I extracted the broadband, household income, wage and state columns spefically.'''
    data = requests.get(url)
    table = data.json()
    

    column_1 = []
    column_2 = []
    column_3 = []
    column_4 = []
    column_5 = []
    column_6 = []
    column_7 = []
    
    for item in table:
        column_1.append(item[0])
        column_2.append(item[1])
        column_3.append(item[2])
        column_4.append(item[3])
        column_5.append(item[4])
        column_6.append(item[5])
        column_7.append(item[6])

    df = pd.DataFrame()
    df[column_1[0]] = column_1[1:]
    df[column_2[0]] = column_2[1:]
    df[column_3[0]] = column_3[1:]
    df[column_4[0]] = column_4[1:]
    df[column_5[0]] = column_5[1:]
    df[column_6[0]] = column_6[1:]
    df[column_7[0]] = column_7[1:]

    return df

# running the function on link retrieved from the census website
df = get_df('https://api.census.gov/data/2018/acs/acs1/pums?get=HINCP,ST,HISPEED&PAP=0&PAP=4:30000&GRPIP=101&GRPIP=1:100&FS=1&FS=2&FES=1&FES=2&FES=3&FES=4&FES=5&FES=6&FES=7&FES=8&PAP=0&PAP=4:30000&GRPIP=101&GRPIP=1:100')

In [19]:
print(df.head())

   HINCP  ST HISPEED PAP GRPIP FS FES
0  83700  26       0   0    14  2   1
1  83700  26       0   0    14  2   1
2  83700  26       0   0    14  2   1
3  83700  26       0   0    14  2   1
4  59000  18       1   0    17  2   4


In [20]:
convert_dict = {
    'PAP':int,  
    'HINCP':int,
    'ST':int,
    'FS': int,
    'GRPIP':int,
    'FES': int,
    'HISPEED':int
}

clean_df = df.astype(convert_dict)

In [21]:
clean_df=clean_df[clean_df['ST'] == 36]

In [22]:
print(clean_df.head())

     HINCP  ST  HISPEED  PAP  GRPIP  FS  FES
39     390  36        0  390    101   1    8
76  106600  36        1    0     14   2    8
77  106600  36        1    0     14   2    8
91   80000  36        1    0     27   2    5
92   80000  36        1    0     27   2    5


In [23]:
# dropping the state column because it is no longer necessary for testing
clean_df.drop(columns='ST', inplace=True)

In [24]:
# replacing 2 values with 0 to follow True or False values
clean_df['FS'].replace(2, 0 , inplace=True)

In [25]:
print(clean_df.head())

     HINCP  HISPEED  PAP  GRPIP  FS  FES
39     390        0  390    101   1    8
76  106600        1    0     14   0    8
77  106600        1    0     14   0    8
91   80000        1    0     27   0    5
92   80000        1    0     27   0    5


In [26]:
FES_dict ={
    1: 2,
    2: 1,
    3: 1,
    4: 0,
    5: 1,
    6: 0,
    7: 1,
    8: 0
}
clean_df.replace(FES_dict, inplace=True)

In [27]:
print(clean_df.sample(15))

         HINCP  HISPEED  PAP  GRPIP  FS  FES
168698  122100        2    0     19   2    2
21008    20000        1    0    101   0    1
40705   167000        0    0     19   0    2
272992   20600        2    0     41   2    1
49241    58320        2    0     29   2    1
155351  300000        2    0     14   0    1
185913   21600        1    0     91   2    1
120213   50000        2    0     28   0    2
287199   65000        2    0     32   2    1
11459    21060        0  400     70   2    1
45773   291340        2    0     12   0    1
37208    38800        2    0     43   2    2
255188   56810        1    0     33   0    0
123342   88000        2    0     19   2    1
209291   40000        2    0     41   2    2


In [33]:
clean_df.to_csv('PUMS_multiple_var.csv')